<a href="https://colab.research.google.com/github/chalsall/colab_reverse_ssh/blob/master/reverse_ssh_tunnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reverse SSH Tunnels for Colab

This Notebook can be used to set up a reverse SSH tunnel from a Google Colaboratory instance out to a properly configured public facing server.  This then allows remote SSH sessions back into said instance for "rich interactive shell" (Bash) access.

> **Important:**  This series of steps -- which results in allowing remote shell access -- is intended to be used to facilitiate software development and deployment testing, which is often much more easily done when a full Bash shell (and all the associated tools) are available to the developer.  ***Please do not abuse this capability.***

Notes:

*   This Notebook assumes a moderate level of proficiency with Linux command-line operations.

*   Shell log-in access to a "public facing" Linux server with a routable IP and an open SSH port is required.

*   Run each section below in order.

---

**This Notebook is Copyright 2019 Chris Halsall (chalsall).  All rights reserved.**

Released under the GPL license version 3.0 or later.  http://www.gnu.org/copyleft/gpl.html

Part of the GPU to 72 Project.  https://www.gpu72.com/  Where geeks do useless things because they make great driving problems...

**This software is provided with no warrantees.  Use at your own risk.**


# Section 1: Generate keys for outgoing Reverse Tunnel.
Generates the public/private keys for the outgoing SSH reverse tunnel.

After running this section, the user **must next copy-and-paste the contents** of .ssh/id_rsa.pub (the last line printed when this section is run; starts with "`ssh-rsa`") into an appropriately configured Linux server with a "public facing" IP address, and an exposed (read: non-firewalled) SSH port (non-standard is, of course, recommended).

Place the contents into the **"`.ssh/authorized_keys`"** file of a non-privialged user's home directory.  Ensure the file permissions are correct (**"`chmod go-rwx`"** for the **"`.ssh`"** directory, and **"`chmod go-wx`"** for the **"`authorized_keys`"** file).  Note this file can contain multiple keys, allowing for mutiple reverse tunnels from mutiple Colab instances.

In [0]:
%cd ~

# Used later, but might as well do this now.  Change the password if so desired (no real security advantage to do so).
!echo -e "linux\nlinux" | passwd root

!rm -rf ssh .ssh
!mkdir .ssh
!chmod go-rwx .ssh
!ln -s .ssh ssh   # Interestingly, the Files viewer to the left won't show files starting with a "."

!ssh-keygen -t rsa -f .ssh/id_rsa -N ""

!ls -lah .ssh
!echo
!echo "Copy and paste the following line into the tunnel target server."
!cat .ssh/id_rsa.pub

# Section 2: Bring up Reverse Tunnel

After running this section (with appropriate changes, as detailed below), a "reverse ssh tunnel" should be created between the local Colab instance and the public facing server configured appropriately above.

Change the following to be appropriate:
*   [TUNNEL_PORT] -- available port number to be SSHed into from remote server.

*   [USER_NAME] -- Username on remote server with above key in the authorized_keys file.

*   [REMOTE_SERVER] -- Remote SSH Server DNS name (or IP).

*   [REMOTE_SSH_PORT] -- Remote SSH Server's listening SSH port (often 22).

After this successfully runs, you should be able to run "`ps auxw | grep sshd | grep [USERNAME]`" and see an sshd process handling the tunnel traffic.

In [0]:
!ssh -N -T -p [SSH_PORT] -R[TUNNEL_PORT]:localhost:22 [USER_NAME]@[REMOTE_SERVER] -o StrictHostKeyChecking=no -i .ssh/id_rsa </dev/null >/dev/null 2>/dev/null &
!echo "Hopefully tunnelled...  Run `ps` on remote server to confirm."

# Section 3: Build standalone OpenSSH server.
Downloads, builds and installs a standalone OpenSSH server from source.  This takes a few minutes.

Note: This step can be eliminated by only downloading a precompiled server from a public facing server.  But this demonstrates how to do it from the definitive sourse.

In [0]:
# Adds the needed user and group for the SSH server.
!groupadd -g 99 sshd
!useradd -u 99 -g 99 -c sshd -d / sshd
!cat /etc/passwd | grep sshd
!cat /etc/group | grep sshd

%cd ~
!mkdir ~/openssh
%cd ~/openssh
!wget -qO openssh.tar.gz https://mirror.csclub.uwaterloo.ca/pub/OpenBSD/OpenSSH/portable/openssh-8.0p1.tar.gz
!tar -xzf openssh.tar.gz
%cd openssh-8.0p1
#!autoconf
!./configure
!make
!make install

!echo -e "\n\nBuild and install complete.\n\n"

!echo "PermitRootLogin yes" >> /usr/local/etc/sshd_config
!cat /usr/local/etc/sshd_config #  For a human review of the config file...

# Section 4: Launch standalone OpenSSH server.

Running this section launches the OpenSSH server, which detaches from the console to become a "daemon".

If you're encountering difficulties in the next section (#5), try adding the "-ddd" options to the server for deep debugging output.

In [0]:
!which sshd
!/usr/local/sbin/sshd # -ddd

#Section 5: Remotely log into this instance.

Last steps:

*   From the command line of the SSH reverse tunnel server, run **"`ssh -p [TUNNEL_PORT] root`"**.  The user on the remote machine running this command does not have to be the same as that used to accept the SSH tunnel connection.

*   You should be prompted with **"`root@localhost's password:`"**.  Enter **"linux"** (or whatever you set the password to be above).

*   You should next be greeted with a bash shell command-line prompt.

**Note that unless debugging is enabled in Section 4, this Notebook should be free to have additional sessons added and run as desired, just like a regular Notebook.**

***The only difference is you also have a fully interactive Bash shell to also work with at the same time...***